# e-Govの法令APIからデータを取って何か面白いことできるのではないかという謎のAPI連携とデータ分析の一石二鳥企画を考えていた件

オープンAPIを使えばいろいろなデータを使えるため分析の幅が広がりネタも
できるのではないかというよくある謎の思い付き提案に振り回される前に試行錯誤をしてみたノートブックファイル

### 参考URL
- https://qiita.com/kzuzuo/items/4e9ee05fc0f41ab188ac#

In [1]:
import requests
import xml.etree.ElementTree as ET

#e-Gov 法令API 仕様書
#https://www.e-gov.go.jp/elaws/pdf/houreiapi_shiyosyo.pdf

#法令選択
def lawName2No(KEYWORD, TAG='LawName'):
    url = 'https://elaws.e-gov.go.jp/api/1/lawlists/2'
    r = requests.get(url)
    root = ET.fromstring(r.content.decode(encoding='utf-8'))
    iflag = 0
    for i,e in enumerate(root.getiterator()):
        if TAG==e.tag:  
            iflag = 0
        if KEYWORD==e.text: 
            iflag = 1
        if iflag==1 and e.tag=='LawNo':
            return  str(e.text)

#法令表示・抽出
def lawTextExtractor(KEYWORD, lawnum='all', wordn=0, words=[]):
    if lawnum=='all':
        url = 'https://elaws.e-gov.go.jp/api/1/lawdata/'+lawName2No(KEYWORD)
    else:
        url = 'https://elaws.e-gov.go.jp/api/1/articles;lawNum='+lawName2No(KEYWORD)+';article='+str(lawnum)
    r = requests.get(url)
    if r.status_code!=requests.codes.ok:
        return
    root = ET.fromstring(r.content.decode(encoding='utf-8')) 
    lawstrs = []
    ssentcount = 0
    for i,e in enumerate(root.getiterator()):
        for xtag in ['LawTitle', 'SupplProvisionLabel', 'ArticleCaption', 'ArticleTitle', 'ParagraphNum', 'ItemTitle', 'Sentence']:
            if e.tag==xtag :

                if wordn==1 and xtag=='Sentence':#ワード指定する場合
                    tmpstr = ''
                    for word in words:
                        if str(e.text).find(word)>-1:
                            if tmpstr=='':
                                tmpstr = e.text.replace(word, '●'+word)
                            else:
                                tmpstr = tmpstr.replace(word, '★'+word)
                    if tmpstr!='':
                        lawstrs.append(tmpstr)
                        ssentcount += 1

                else:
                    lawstrs.append(str(e.text))
    if lawstrs==[]:
        pass
    else:
        if lawnum=='all':
            print('●条項号数', ssentcount)
        print('\n'.join(lawstrs))

In [4]:
#法令指定
KEYWORD = '道路交通法'

#ワード抽出する場合　1
wordn = 1 
#抽出ワードリスト
words = ['車'] 

#条文指定 rangeの値が条文の範囲を表す
#range(1,10)であれば、条文１～10項
for lawnum in range(1,10): 
    lawTextExtractor(KEYWORD, lawnum, wordn, words)
    #「条の～」を無理やり処理．後で対策を考える．
    [lawTextExtractor(KEYWORD, str(lawnum)+'.'+str(i), wordn, words) for i in range(15)]

#法令全体指定する場合（3秒ほどで出力される）
a = lawTextExtractor(KEYWORD, 'all', wordn, words)

（目的）
第一条
None
（定義）
第二条
None
一
道路法（昭和二十七年法律第百八十号）第二条第一項に規定する道路、道路運送法（昭和二十六年法律第百八十三号）第二条第八項に規定する自動●車道及び一般交通の用に供するその他の場所をいう。
二
三
●車道
●車両の通行の用に供するため縁石線若しくはさくその他これに類する工作物又は道路標示によつて区画された道路の部分をいう。
三の二
本線●車道
高速自動●車国道（高速自動●車国道法（昭和三十二年法律第七十九号）第四条第一項に規定する道路をいう。以下同じ。）又は自動●車専用道路（道路法第四十八条の四に規定する自動●車専用道路をいう。以下同じ。）の本線●車線により構成する●車道をいう。
三の三
自転●車道
自転●車の通行の用に供するため縁石線又はさくその他これに類する工作物によつて区画された●車道の部分をいう。
三の四
歩行者の通行の用に供し、又は●車道の効用を保つため、歩道の設けられていない道路又は道路の歩道の設けられていない側の路端寄りに設けられた帯状の道路の部分で、道路標示によつて区画されたものをいう。
四
四の二
自転●車横断帯
道路標識等により自転●車の横断の用に供するための場所であることが示されている道路の部分をいう。
五
十字路、丁字路その他二以上の道路が交わる場合における当該二以上の道路（歩道と●車道の区別のある道路においては、●車道）の交わる部分をいう。
六
路面電●車に乗降する者若しくは横断している歩行者の安全を図るため道路に設けられた島状の施設又は道路標識及び道路標示により安全地帯であることが示されている道路の部分をいう。
七
●車両通行帯
●車両が道路の定められた部分を通行すべきことが道路標示により示されている場合における当該道路標示により示されている道路の部分をいう。
八
●車両
自動●車、原動機付自転●車、軽●車両及びトロリーバスをいう。
九
自動●車
原動機を用い、かつ、レール又は架線によらないで運転する●車であつて、原動機付自転●車、自転●車及び身体障害者用の●車いす並びに歩行補助●車その他の小型の●車で政令で定めるもの（以下「歩行補助●車等」という。）以外のものをいう。
十
原動機付自転●車
内閣府令で定める大きさ以下の総排気量又は定格出力を有する原動機を用い、かつ、レール又は架線によ